# Analysis of JCR Impact Factor

Analysis of impact factor of publication in specific field. For remote sensing the data can be obtained with:

https://plus.cobiss.si/opac7/jcr?kw=remote%20sensing&max=1000

In [ ]:
# Libraries
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
# Figure size and style
plt.rcParams['figure.figsize'] = (15, 10)

In [ ]:
# Cobiss base URL
cobiss_url = "https://plus.cobiss.si/opac7/jcr?"
cobiss_param = "&max=1000"
# Field
keywoard = "remote sensing"

In [ ]:
# Read data
cobiss_jcr = cobiss_url + "kw=" + keywoard + cobiss_param
cobiss_jcr = cobiss_jcr.replace(" ", "%20") # replace spaces
jcr_if = pd.read_html(cobiss_jcr)[0] # Take first table

In [ ]:
jcr_if = pd.read_html(cobiss_jcr)[0] # Take first table
jcr_if.head()

In [ ]:
jcr_if.describe()

In [ ]:
# List of journals
jcr_if_journals = jcr_if[["ISSN", "Naslov serijske publikacije"]]
jcr_if_journals = jcr_if_journals.drop_duplicates().reset_index(drop=True)
jcr_if_journals

In [ ]:
# Top 10 journals in last year
ylast = jcr_if["Leto"].max()
jcr_if_ylast = jcr_if[jcr_if["Leto"] == ylast]
jcr_if_ylast.nlargest(10, 'Faktor vpliva')[["Naslov serijske publikacije", "ISSN"]]

In [ ]:
jcr_if_top_ylast = jcr_if_ylast.nlargest(10, 'Faktor vpliva')["ISSN"]
jcr_if_top_ylast

In [ ]:
# Drop unneeded columns
jcr_if_issn = jcr_if[["ISSN", "Leto", "Faktor vpliva"]]

In [ ]:
# Select only top publications
jcr_if_top = jcr_if_issn[jcr_if_issn['ISSN'].isin(jcr_if_top_ylast)]
jcr_if_top.head()

In [ ]:
# Group by publication ISSN
jcr_if_top_grouped = jcr_if_top.groupby(["Leto", "ISSN"]).mean().unstack()
jcr_if_top_grouped.columns = jcr_if_top_grouped.columns.get_level_values(1)

In [ ]:
jcr_if_top_grouped

In [ ]:
# Plot IF by year
jcr_if_top_grouped.plot()

In [ ]:
jcr_if_top_grouped_columns = jcr_if_top_grouped.columns

In [ ]:
df.loc[df['B'] == 3, 'A'].iloc[0]

In [ ]:
jcr_if_journals.loc[jcr_if_journals["ISSN"] == "0034-4257", "Naslov serijske publikacije"].iloc[0]

In [ ]:
for idx, val in enumerate(jcr_if_top_grouped.columns):
    print(idx, val)
    jcr_if_top_grouped.columns[idx] = 
    jcr_if_journals.loc[jcr_if_journals["ISSN"] == val, "Naslov serijske publikacije"].iloc[0]

In [ ]:
jcr_if_journals[jcr_if_journals["ISSN"] == jcr_if_top_grouped.columns[1]]["Naslov serijske publikacije"]